## algorithm design and anlysis-2025 spring  homework 4
**Deadline**：2025.5.14

**name**:


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

## 问题 1 
**最小生成树（Minimum Spanning Tree）**

设  **G**  为一个带权重的连通无向图，且所有边的权重均不相等。令$e_i$ 为权重第 $i$ 小的边。最小生成树（MST）是否必须包含 $e_1$ ? 同理，是否必须包含 $e_2$ 和 $e_3$ ? 若必须包含，请给出证明；否则，请构造反例。需从基本原理论证，不能依赖割引理(cut lemma) 或 Prim/Kruskal算法的正确性。


answer:

## 问题 2 
**瓶颈生成树（Bottleneck Spanning Tree）**

带有权重的无向图 $G(V,E,w)$ 的瓶颈生成树，表现为：在所有生成树中，最大权重边的权重值最小。即，BST $T$ 最小化瓶颈损失 $c(T)=max_{e \in T}{w(e)}$。

1. 证明 $G$ 的每一个最小生成树（MST）都是瓶颈生成树（BST）
2. 设计一个线性时间复杂度的算法：， 对于一个图 $G(V,E,w)$ 和一个整数 $b$，判断图 $ G$ 是否存在一个瓶颈生成树，其最大权重边的权重不超过 $b$，分析算法设计思路，并基于python编程实现。
3. 设计一个线性时间复杂度的算法：对于给定的图 $G(V,E,w)$，找到其瓶颈生成树，分析算法设计思路，并基于python编程实现。

idea：

In [ ]:
#问题2_2
#定义图的边结构
class Edge:
    def __init__(self, u, v, w):
        self.u = u
        self.v = v
        self.w = w

def has_bst_with_limit(graph, b):
    vertices = set()
    for edge in graph:
        vertices.add(edge.u)
        vertices.add(edge.v)
    n = len(vertices)
    visited = {vertex: False for vertex in vertices}

    new_graph = {vertex: [] for vertex in vertices}
    valid_edges = []
    for edge in graph:
        if edge.w <= b:
            new_graph[edge.u].append(edge.v)
            new_graph[edge.v].append(edge.u)
            valid_edges.append(edge)

    def dfs(vertex):
        visited[vertex] = True
        for neighbor in new_graph[vertex]:
            if not visited[neighbor]:
                dfs(neighbor)

    #从第一个顶点开始进行深度优先搜索
    start_vertex = next(iter(vertices))
    dfs(start_vertex)

    #检查是否所有顶点都被访问过
    result = all(visited.values())
    if result:
        edge_info = [(edge.u, edge.v, edge.w) for edge in valid_edges]
        print("存在最大权重不超过{}的瓶颈生成树.".format(b))
        print("用于构建瓶颈生成树的边为：", edge_info)
    else:
        print("不存在最大权重不超过{}的瓶颈生成树.".format(b))
    return result


#示例图1:存在满足条件的瓶颈生成树
graph1 = [Edge(0, 1, 3), Edge(1, 2, 2), Edge(2, 3, 4), Edge(3, 0, 3)]
b1 = 4
print("示例图1的结果:")
has_bst_with_limit(graph1, b1)

# 示例图2:不存在满足条件的瓶颈生成树
graph2 = [Edge(0, 1, 5), Edge(1, 2, 6), Edge(2, 3, 7), Edge(3, 0, 8)]
b2 = 4
print("\n示例图2的结果:")
has_bst_with_limit(graph2, b2)
#算法设计思路:利用深度优先搜索(DFS)或广度优先搜索(BFS)来遍历图,在遍历过程中,只考虑权重不超过b的边.如果通过这些边能够遍历到图中的所有顶点,
#说明存在一个生成树,且这个生成树的最大权重边不超过b,即存在满足条件的瓶颈生成树;否则不存在.
#时间复杂度分析:无论是DFS还是BFS,对图进行遍历,每个顶点和边最多被访问一次,时间复杂度为O(V + E),V是顶点数,E是边数,符合线性时间复杂度要求.

#问题2_3:
class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)
        if root_x != root_y:
            if self.rank[root_x] > self.rank[root_y]:
                self.parent[root_y] = root_x
            elif self.rank[root_x] < self.rank[root_y]:
                self.parent[root_x] = root_y
            else:
                self.parent[root_y] = root_x
                self.rank[root_x] += 1
            return True
        return False

class Edge:
    def __init__(self, u, v, w):
        self.u = u
        self.v = v
        self.w = w

def find_bottleneck_spanning_tree(graph):
    edges = sorted([Edge(u, v, w) for u, v, w in graph], key=lambda x: x.w)
    n = len(set([u for u, _, _ in graph] + [v for _, v, _ in graph]))
    uf = UnionFind(n)
    mst_edges = []
    for edge in edges:
        if uf.union(edge.u, edge.v):
            mst_edges.append(edge)
            if len(mst_edges) == n - 1:
                break
    if mst_edges:
        bottleneck_weight = max([edge.w for edge in mst_edges])
        mst_edge_info = [(edge.u, edge.v, edge.w) for edge in mst_edges]
        print("找到瓶颈生成树,其最大权重边的权重为:", bottleneck_weight)
        print("构成瓶颈生成树的边为:", mst_edge_info)
        return bottleneck_weight
    else:
        print("未找到瓶颈生成树.")
        return None

#示例
graph = [(0, 1, 2), (1, 2, 3), (0, 2, 4)]
find_bottleneck_spanning_tree(graph)
#算法设计思路:首先对图的边按照权重从小到大进行排序,然后利用并查集(Union-Find)数据结构来逐步构建生成树,
#从权重最小的边开始,每次添加一条边到当前的生成树集合中,只要添加后不形成环,当添加完足够的边(V-1条边,V是顶点数)构成生成树时,此时最后添加的边就是该生成树的瓶颈边,记录下其权重.
#继续这个过程,找到所有可能生成树中瓶颈边权重最小的那个生成树,即为瓶颈生成树.

#时间复杂度分析:对边排序的时间复杂度为O(ElogE),在并查集操作构建生成树过程中,每次并查集操作的均摊时间复杂度接近常数,总共需要添加V-1,
#所以这部分时间复杂度为O(E+V).
#整体时间复杂度为O(ElogE),在稀疏图E=O(V)情况下,可近似看作线性时间复杂度O(VlogV).

## 问题 3

**道路网（Road Network）**

假设有一个以图 $ G(V, E, l) $ 表示的道路网络，连接了一组城市 $ V $。我们假设该网络是有向的，并且每条道路 $(u, v) \in E$ 都有一个非负的长度 $ l(u, v) $。一条新的道路即将被建造，因此有一个列表 $ E' $ 包含它可以连接的城市对。每对 $(u, v) \in E'$ 都有一个对应的长度 $ l'(u, v) $。我们希望选择一对城市，使得两个城市 $ s, t \in V $ 之间的距离减少最大。请为此问题编写一个高效的算法，并详细解释算法的正确性和复杂度。


In [ ]:
import math

def floyd_warshall(graph):
    node_num = len(graph)
    distances = [[graph[i][j] if graph[i][j] != -1 else math.inf for j in range(node_num)] for i in range(node_num)]
    for mid in range(node_num):
        for start in range(node_num):
            for end in range(node_num):
                distances[start][end] = min(distances[start][end], distances[start][mid] + distances[mid][end])
    return distances

def find_best_new_road(graph, potential_roads):
    node_num = len(graph)
    init_distances = floyd_warshall(graph)
    max_reduction = -math.inf
    best_road = None
    for u, v, road_len in potential_roads:
        new_graph = [row[:] for row in graph]
        new_graph[u][v] = road_len
        new_distances = floyd_warshall(new_graph)
        reduction = 0
        for i in range(node_num):
            for j in range(node_num):
                reduction = max(reduction, init_distances[i][j] - new_distances[i][j])
        if reduction > max_reduction:
            max_reduction = reduction
            best_road = (u, v)
    return best_road, max_reduction

graph = [
    [-1, 3, -1],
    [-1, -1, 4],
    [-1, -1, -1]
]
potential_roads = [(0, 2, 2)]
best_road, max_reduction = find_best_new_road(graph, potential_roads)
print("最佳新建道路:", best_road)
print("最大距离减少量:", max_reduction)

#算法正确性:Floyd-Warshall算法基于动态规划通过考虑所有中间节点能正确算出初始所有顶点对最短路径,对E'中每条新建道路计算其对各顶点对路径影响及距离减少量,可找出使距离减少最大的城市对.
#时间复杂度:Floyd-Warshall算法本身时间复杂度为O(|V|^3).对于E'中每对城市,更新距离最坏接近O(|V|^2),处理E'中所有城市对时间复杂度为O(|E'|✖|V|^2),故总时间复杂度为O(|E'|✖|V|^2+|V|^3).
#空间复杂度:需用二维数组存储所有城市对最短距离,数组大小|V|✖|V|,所以空间复杂度为O(|V|^2).

## 问题 4

**逃离问题**

一个 $ n \times n $ 的网格是一个无向图，由 $ n $ 行和 $ n $ 列的顶点组成，如下图所示。我们用 $(i,j)$ 表示第 $ i $ 行和第 $ j $ 列的顶点。除了边界顶点，网格中的所有顶点都有四个邻居，即满足 $ i = 1, i = n, j = 1 $ 或 $ j = n $ 的点 $(i,j)$。

给定网格中的 $ m \leq n^2 $ 个起点 $(x_1, y_1), (x_2, y_2), \cdots , (x_m, y_m)$，逃离问题是确定是否存在 $ m $ 条顶点不相交的路径（即路径之间不相交），从这些起点到边界上的任意 $ m $ 个不同点。例如，图1中的网格存在逃离。

(1) 该问题可以看作是一个最大流问题。考虑一个流网络，其中顶点和边都有容量。也就是说，进入任何给定顶点的总正流量受到容量限制。证明在具有边和顶点容量的网络中确定最大流可以简化为在具有可比大小的普通流网络上的最大流问题。更准确地说，你需要将一个具有顶点和边容量的网络 $ G = (V,E) $ 转换为另一个仅具有边容量的网络 $ G' = (V', E') $，使得两个网络上的最大流相同，并且你构建的新网络具有 $ V' = O(V) $ 个顶点和 $ E' = O(E) $ 条边。你可以假设网络是连通的。

(2) 描述一个解决逃离问题的高效算法，并分析其运行时间。


<div align="center"> <img alt="图片" src="./fig/escepe-p.png"> </div>
<center> 图2. 逃脱问题网格，起始顶点为黑色，其他网格顶点为白色</center>

idea：

In [ ]:
import networkx as nx

def escape_problem(n, start_points):
    #创建有向图
    G = nx.DiGraph()
    #添加源点和汇点
    source = 'S'
    sink = 'T'
    G.add_node(source)
    G.add_node(sink)
    #构建流网络
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            #顶点拆分
            in_node = (i, j, 'in')
            out_node = (i, j, 'out')
            G.add_node(in_node)
            G.add_node(out_node)
            G.add_edge(in_node, out_node, capacity=1)
            #处理邻居边
            if i > 1:
                neighbor_in = (i - 1, j, 'in')
                neighbor_out = (i - 1, j, 'out')
                G.add_edge(out_node, neighbor_in, capacity=1)
            if i < n:
                neighbor_in = (i + 1, j, 'in')
                neighbor_out = (i + 1, j, 'out')
                G.add_edge(out_node, neighbor_in, capacity=1)
            if j > 1:
                neighbor_in = (i, j - 1, 'in')
                neighbor_out = (i, j - 1, 'out')
                G.add_edge(out_node, neighbor_in, capacity=1)
            if j < n:
                neighbor_in = (i, j + 1, 'in')
                neighbor_out = (i, j + 1, 'out')
                G.add_edge(out_node, neighbor_in, capacity=1)
            #边界顶点连接到汇点
            if i == 1 or i == n or j == 1 or j == n:
                G.add_edge(out_node, sink, capacity=1)
    #连接起点到源点
    for start in start_points:
        x, y = start
        in_node = (x, y, 'in')
        G.add_edge(source, in_node, capacity=1)

    #使用Dinic算法计算最大流
    max_flow_value = nx.maximum_flow_value(G, source, sink)
    return max_flow_value == len(start_points)


#示例1
n = 4
start_points = [(1, 1), (2, 2)]
result = escape_problem(n, start_points)
print(f"是否存在逃离方案:{result}")

#示例2
n = 5
start_points = [(2,2), (3,3), (4,4)]
result = escape_problem(n, start_points)
print(f"是否存在逃离方案:{result}")